In [13]:
using Random
using Printf
function modinv_safe(a::Int, m::Int)
    a = mod(a, m)
    t, newt = 0, 1
    r, newr = m, a
    while newr != 0
        q = r ÷ newr
        t, newt = newt, t - q * newt
        r, newr = newr, r - q * newr
    end
    if r != 1
        return nothing
    end
    return mod(t, m)
end
function make_f(p::Int, g::Int, h::Int; partition_type=:mod3)
    order = p - 1
    if partition_type == :mod3
        f = function(X, a, b)
            r = X % 3
            if r == 0
                X = (X * h) % p; b = (b + 1) % order
            elseif r == 1
                X = (X * X) % p; a = (2a) % order; b = (2b) % order
            else
                X = (X * g) % p; a = (a + 1) % order
            end
            return X, a, b
        end
    else
        c1 = rand(1:order); c2 = rand(1:order)
        f = function(X, a, b)
            r = mod(hash(X), 4)
            if r == 0
                X = (X * h) % p; b = (b + c1) % order
            elseif r == 1
                X = (X * X) % p; a = (2a) % order; b = (2b) % order
            elseif r == 2
                X = (X * g) % p; a = (a + c2) % order
            else
                X = (X * (g*h % p)) % p; a = (a + 1) % order; b = (b + 1) % order
            end
            return X, a, b
        end
    end
    return f
end
function pollard_rho_dlog(g::Int, h::Int, p::Int; max_iter::Int=10^6, max_restarts::Int=20, partition_type=:mod3, verbose::Bool=false)
    order = p - 1
    for attempt in 1:max_restarts
        a = rand(0:order-1)
        b = rand(0:order-1)
        X = (powermod(g, a, p) * powermod(h, b, p)) % p
        A, B, Y = a, b, X
        f = make_f(p, g, h; partition_type=partition_type)
        if verbose
            @printf("Попытка %d: старт a=%d b=%d X=%d\n", attempt, a, b, X)
        end
        for iter in 1:max_iter
            X, a, b = f(X, a, b)
            Y, A, B = f(Y, A, B)
            Y, A, B = f(Y, A, B)
            if X == Y
                r = mod(b - B, order)
                lhs = mod(A - a, order)
                if r == 0
                    if verbose
                        @printf("Итерация %d: r=0, перезапуск.\n", iter)
                    end
                    break
                end
                invr = modinv_safe(r, order)
                if invr === nothing
                    if verbose
                        @printf("Итерация %d: нет обратного для r=%d, перезапуск.\n", iter, r)
                    end
                    break
                end
                x_candidate = mod(lhs * invr, order)
                if powermod(g, x_candidate, p) == h % p
                    if verbose
                        @printf("Найдено решение: x=%d\n", x_candidate)
                    end
                    return x_candidate
                else
                    if verbose
                        @printf("x=%d не подошёл, перезапуск.\n", x_candidate)
                    end
                    break
                end
            end
        end
    end
    error("Не удалось найти дискретный логарифм после $max_restarts попыток.")
end
p = 1024
g = 2
x_true = 123
h = powermod(g, x_true, p)
x_found = pollard_rho_dlog(g, h, p; max_iter=10^5, max_restarts=50, partition_type=:hash, verbose=true)
@printf("Результат: найден x = %d (истинный x = %d)\n", x_found, x_true)

Попытка 1: старт a=686 b=172 X=0
Итерация 1: нет обратного для r=477, перезапуск.
Попытка 2: старт a=501 b=558 X=0
Итерация 1: нет обратного для r=741, перезапуск.
Попытка 3: старт a=837 b=1005 X=0
x=0 не подошёл, перезапуск.
Попытка 4: старт a=105 b=647 X=0
Итерация 1: нет обратного для r=267, перезапуск.
Попытка 5: старт a=802 b=806 X=0
Итерация 1: нет обратного для r=186, перезапуск.
Попытка 6: старт a=560 b=75 X=0
x=0 не подошёл, перезапуск.
Попытка 7: старт a=265 b=398 X=0
x=0 не подошёл, перезапуск.
Попытка 8: старт a=91 b=995 X=0
x=0 не подошёл, перезапуск.
Попытка 9: старт a=937 b=315 X=0
Итерация 1: нет обратного для r=407, перезапуск.
Попытка 10: старт a=702 b=851 X=0
x=0 не подошёл, перезапуск.
Попытка 11: старт a=397 b=927 X=0
x=0 не подошёл, перезапуск.
Попытка 12: старт a=368 b=768 X=0
Итерация 1: нет обратного для r=713, перезапуск.
Попытка 13: старт a=263 b=974 X=0
Итерация 1: нет обратного для r=696, перезапуск.
Попытка 14: старт a=583 b=752 X=0
x=0 не подошёл, перезап

LoadError: Не удалось найти дискретный логарифм после 50 попыток.